In [1]:
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt

from extraction import extract_patches
from reconstruction import perform_voting, generate_indexes

from model import Multimodel

Using TensorFlow backend.


In [15]:
import os
import nibabel as nib

from medical_data import cdr_info, nwbv_info, diff_info

orig_pattern = 'datasets/OASIS/OAS2_RAW_PART1/OAS2_00{0:02}_MR{1}/RAW/mpr-1_sstr_susan_matched.nii.gz'
prob_pattern = 'datasets/OASIS/OAS2_RAW_PART1/OAS2_00{0:02}_MR{1}/RAW/mpr-1_sstr_susan_pve_{2}.nii.gz'

step = (32, 32)
threshold = np.int32(0.50 * np.prod(curr_patch_shape[:]))
seg_train = np.empty((0, 1, ) + curr_patch_shape)
ref_train = np.empty((0, 2, ) + curr_patch_shape)
out_train = np.empty((0, 1, ) + curr_patch_shape)
for i in np.append(range(1, 76), range(80, 100)) :
    print '{} :'.format(i),
    
    for j in range(1, 6) :
        ref_filename = orig_pattern.format(i, j)

        if not os.path.exists(ref_filename) :
            continue
        
        is_last_scan = True
        for k in range(j+1, 6) :
            mov_filename = orig_pattern.format(i, k)

            if os.path.exists(mov_filename) :
                is_last_scan = False
                break
                
        if is_last_scan :
            break
            
        if diff_info['OAS2_00{0:02}_MR{1}'.format(i, k)] == 0 :
            break

        volume_init = nib.load(ref_filename).get_data()

        mask_patches = extract_patches(volume_init != 0, (1, ) + curr_patch_shape, (1, ) + step)

        useful_patches = np.sum(mask_patches, axis=(1, 2, 3)) > threshold

        N = np.sum(useful_patches)

        volume_init = (volume_init - volume_init.mean()) / volume_init.std()

        ref_patches = extract_patches(volume_init, (1, ) + curr_patch_shape, (1, ) + step)
        ref_patches = ref_patches[useful_patches].reshape((-1, 1, ) + curr_patch_shape)

        new_patches = np.empty((N, 1, ) + curr_patch_shape)
        for a in range(1) :
            prob_filename = prob_pattern.format(i, j, a)
            prob_mask_init = nib.load(prob_filename).get_data()
            prob_patches = extract_patches(prob_mask_init, (1, ) + curr_patch_shape, (1, ) + step)
            prob_patches = prob_patches[useful_patches].reshape((-1, ) + curr_patch_shape)

            new_patches[:, a] = prob_patches

            del prob_patches
        
        ref_train = np.vstack((np.hstack((ref_patches, new_patches)), ref_train)) ##
        del volume_init, ref_patches, new_patches

        #####################
        volume_init = nib.load(mov_filename).get_data()
        volume_init = (volume_init - volume_init.mean()) / volume_init.std()

        mov_patches = extract_patches(volume_init, (1, ) + curr_patch_shape, (1, ) + step)
        mov_patches = mov_patches[useful_patches].reshape((-1, 1, ) + curr_patch_shape)

        out_train = np.vstack((mov_patches, out_train)) ##
        del volume_init, mov_patches

        new_patches = np.empty((N, 1, ) + curr_patch_shape)
        for a in range(1) :
            prob_filename = prob_pattern.format(i, k, a)
            prob_mask_init = nib.load(prob_filename).get_data()
            prob_patches = extract_patches(prob_mask_init, (1, ) + curr_patch_shape, (1, ) + step)
            prob_patches = prob_patches[useful_patches].reshape((-1, ) + curr_patch_shape)

            new_patches[:, a] = prob_patches

            del prob_patches

        seg_train = np.vstack((new_patches, seg_train)) ##
        del new_patches

    print

1 :
2 :
3 :
4 :
5 :
6 :
7 :
8 :
9 :
10 :
11 :
12 :
13 :
14 :
15 :
16 :
17 :
18 :
19 :
20 :
21 :
22 :
23 :
24 :
25 :
26 :
27 :
28 :
29 :
30 :
31 :
32 :
33 :
34 :
35 :
36 :
37 :
38 :
39 :
40 :
41 :
42 :
43 :
44 :
45 :
46 :
47 :
48 :
49 :
50 :
51 :
52 :
53 :
54 :
55 :
56 :
57 :
58 :
59 :
60 :
61 :
62 :
63 :
64 :
65 :
66 :
67 :
68 :
69 :
70 :
71 :
72 :
73 :
74 :
75 :
80 :
81 :
82 :
83 :
84 :
85 :
86 :
87 :
88 :
89 :
90 :
91 :
92 :
93 :
94 :
95 :
96 :
97 :
98 :
99 :


In [19]:
curr_patch_shape = (128, 128)
input_modalities = ['T1', 'seg']
output_modalities = ['Gen']
output_weights = {'Gen' : 1.0, 'concat' : 1.0}
latent_dim = 32
channels = [2, 1]
use_dropout = [False, False]
patch_shape = curr_patch_shape
scale = 1
a_model = Multimodel(
    input_modalities, output_modalities, output_weights, latent_dim, channels, patch_shape, use_dropout, scale)
a_model.build()

Latent dimensions: 32
Fuse latent representations using max
making output: Tensor("enc_T1_act9_5/add:0", shape=(?, 32, 128, 128), dtype=float32) Tensor("dec_Gen_15/dec_Gen_act5/add:0", shape=(?, 1, 128, 128), dtype=float32) em_0_dec_Gen
making output: Tensor("enc_seg_act9_5/add:0", shape=(?, 32, 128, 128), dtype=float32) Tensor("dec_Gen_16/dec_Gen_act5/add:0", shape=(?, 1, 128, 128), dtype=float32) em_1_dec_Gen
making output: Tensor("combined_em_5/Maximum:0", shape=(?, 32, 128, 128), dtype=float32) Tensor("dec_Gen_17/dec_Gen_act5/add:0", shape=(?, 1, 128, 128), dtype=float32) em_2_dec_Gen
making output: em_concat Tensor("em_concat_5/concat:0", shape=(?, 2, ?), dtype=float32) em_concat_5/concat:0
all outputs:  [u'em_0_dec_Gen_5/add:0', u'em_1_dec_Gen_5/add:0', u'em_2_dec_Gen_5/add:0', u'em_concat_5/concat:0', u'em_fused_5/ExpandDims:0']
output dict:  {'em_concat': <function embedding_distance at 0x7f27ef5c6aa0>, 'em_fused': <function embedding_distance at 0x7f27ef5c6aa0>, 'em_1_dec_Gen'

In [20]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

patience = 2

stopper = EarlyStopping(patience=patience)
checkpointer = ModelCheckpoint('model.h5', save_best_only=True, save_weights_only=True)

N = len(ref_train)
a_model.model.fit(
    [ref_train, seg_train],
    [out_train, out_train, out_train, np.empty((N, 2, 0)), np.empty((N, 1, 0))],
    validation_split=0.3, epochs=20,
    callbacks=[checkpointer, stopper])

Train on 6493 samples, validate on 2783 samples
Epoch 1/20
6493/6493 [==============================] - 111s 17ms/step - loss: 0.8180 - em_0_dec_Gen_loss: 0.2018 - em_1_dec_Gen_loss: 0.3972 - em_2_dec_Gen_loss: 0.1996 - em_concat_loss: 0.0195 - em_fused_loss: 0.0000e+00 - val_loss: 0.4846 - val_em_0_dec_Gen_loss: 0.1419 - val_em_1_dec_Gen_loss: 0.2087 - val_em_2_dec_Gen_loss: 0.1234 - val_em_concat_loss: 0.0107 - val_em_fused_loss: 0.0000e+00
Epoch 2/20
6493/6493 [==============================] - 96s 15ms/step - loss: 0.4436 - em_0_dec_Gen_loss: 0.1388 - em_1_dec_Gen_loss: 0.1801 - em_2_dec_Gen_loss: 0.1165 - em_concat_loss: 0.0082 - em_fused_loss: 0.0000e+00 - val_loss: 0.4468 - val_em_0_dec_Gen_loss: 0.1380 - val_em_1_dec_Gen_loss: 0.1884 - val_em_2_dec_Gen_loss: 0.1139 - val_em_concat_loss: 0.0064 - val_em_fused_loss: 0.0000e+00
Epoch 3/20
6493/6493 [==============================] - 96s 15ms/step - loss: 0.4027 - em_0_dec_Gen_loss: 0.1347 - em_1_dec_Gen_loss: 0.1568 - em_2_dec_Gen

In [21]:
a_model.model.load_weights('model.h5')

In [24]:
from keras.models import Model
import os
import nibabel as nib
import SimpleITK as sitk

orig_pattern = 'datasets/OASIS/OAS2_RAW_PART1/OAS2_00{0:02}_MR{1}/RAW/mpr-1_sstr_susan_matched.nii.gz'
prob_pattern = 'datasets/OASIS/OAS2_RAW_PART1/OAS2_00{0:02}_MR{1}/RAW/mpr-1_sstr_susan_pve_{2}.nii.gz'

step = (32, 32)
for i in range(76, 80) :
    ref_filename = orig_pattern.format(i, 1)
    mov_filename = orig_pattern.format(i, 3)
    if not (os.path.exists(ref_filename) and os.path.exists(mov_filename)) :
        continue
    
    print i

    volume_data = nib.load(ref_filename)
    volume_actual = volume_data.get_data()
    volume_actual = (volume_actual - volume_actual.mean()) / volume_actual.std()
    ref_patches = extract_patches(volume_actual, (1, ) + curr_patch_shape, (1, ) + step)
    ref_patches = ref_patches.reshape((-1, 1, ) + curr_patch_shape)
    
    N = len(ref_patches)

    old_patches = np.empty((N, 1, ) + curr_patch_shape)
    for k in range(1) :
        filename = prob_pattern.format(i, 1, k)
        prob_mask_init = nib.load(filename).get_data()
        prob_patches = extract_patches(prob_mask_init, (1, ) + curr_patch_shape, (1, ) + step)
        prob_patches = prob_patches.reshape((-1, ) + curr_patch_shape)

        old_patches[:, k] = prob_patches

        del prob_patches
    
    new_patches = np.empty((N, 1, ) + curr_patch_shape)
    for k in range(1) :
        filename = prob_pattern.format(i, 3, k)
        prob_mask_init = nib.load(filename).get_data()
        prob_patches = extract_patches(prob_mask_init, (1, ) + curr_patch_shape, (1, ) + step)
        prob_patches = prob_patches.reshape((-1, ) + curr_patch_shape)

        new_patches[:, k] = prob_patches

        del prob_patches
    
    pred = a_model.model.predict([np.hstack((ref_patches, old_patches)), new_patches], verbose=1)[2]

    volume = perform_voting(pred.reshape((-1, 1, ) + curr_patch_shape), (1, ) + curr_patch_shape, (256, 256, 128), (1, ) + step)
    
    volume_data = nib.load(mov_filename)
    volume_actual = volume_data.get_data()
    volume = volume * volume_actual.std() + volume_actual.mean()
    
    volume = np.multiply(volume_data.get_data() != 0, volume)

    nib.save(nib.Nifti1Image(volume, volume_data.affine), 'result_{}.nii.gz'.format(i))
    
    del new_patches, ref_patches
    
    res = sitk.ReadImage('result_{}.nii.gz'.format(i))

    caster = sitk.CastImageFilter()
    caster.SetOutputPixelType(res.GetPixelID())

    orig = caster.Execute(sitk.ReadImage(orig_pattern.format(i, 1)))

    matcher = sitk.HistogramMatchingImageFilter()
    matcher.SetNumberOfHistogramLevels(1024)
    matcher.SetNumberOfMatchPoints(15)
    matched = matcher.Execute(res, orig)

    sitk.WriteImage(matched, 'result_cor_{}.nii.gz'.format(i))

76
1280/1280 [==============================] - 10s 8ms/step
78
1280/1280 [==============================] - 10s 8ms/step
79
1280/1280 [==============================] - 10s 8ms/step
